## Init

Business layer :why sql is preferred over PySpark in business layer

## Read From Silver Table/s

In [0]:
%sql SELECT * FROM workspace.silver.erp_customers

In [0]:
%sql SELECT * FROM workspace.silver.crm_cust_info

In [0]:
%sql
SELECT * FROM workspace.silver.erp_customer_location

## Business Transformation and Modeling

In [0]:
query = """
SELECT
    ROW_NUMBER() OVER (ORDER BY ci.customer_key) AS customer_rank,
    ci.customer_key,
    ci.firstname,
    ci.lastname,
    la.country,
    ci.marital_status,
    CASE
        WHEN ci.gender <> 'n/a' THEN ci.gender
        ELSE COALESCE(ca.gender, 'n/a')
    END AS gender,
    ca.birth_date AS birthdate,
    ci.create_date AS create_date
FROM silver.crm_cust_info ci
LEFT JOIN silver.erp_customers ca
    ON ci.customer_key = ca.customer_number
LEFT JOIN silver.erp_customer_location la
    ON ci.customer_key = la.customer_number
"""
df = spark.sql(query)


In [0]:
df.display()

## Write it ti Gold Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.gold.dim_customers")

In [0]:
%sql
SELECT * FROM workspace.gold.dim_customers